In [ ]:
import numpy as np
import pandas as pd

var_CF = "CF"
var_BS = "BS"
var_IS = "IS"
var_FS = var_IS



folder_mai = "edgarfiles/"
folder_pas = "common files"
folder_fel = "this folder has the quarterly and annual files"
folder_mai = "common files are here"
folder_ann = "this folder has three subfodler CF, BS, IS for the three financial statements"

In [ ]:

# Get all Entries
df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str,"ffy":str,"line":str}
df_ANNUAL = pd.read_csv((folder_fel+"ANNUAL ALL.csv.zip"),dtype=df_col_types)
df_ANNUAL.drop(["Unnamed: 0"],axis=1,inplace=True)
df_ANNUAL["tag"] = df_ANNUAL["tag"].str.upper()
print(len(df_ANNUAL))

df_currency = pd.read_excel((folder_mai+"Currency Data.xlsx"),sheet_name="subset")

# GET ALL TAGS
df_all_tags = pd.read_excel((folder_mai+"Tags and Stats.xlsx"),sheet_name="Tag List")
df_all_tags.sort_values(by=["stmt","metric","tag_order"],ascending=True,inplace=True)
print(len(df_all_tags))

# GET THE INDUSTRY AND THE SECTOR
df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str}
df_cik_sic_align = pd.read_csv((folder_mai+"cik_sic_align.csv"),dtype=df_col_types)
df_sic_data = df_cik_sic_align[["sic","Office","Industry Title"]] 
df_sic_data.drop_duplicates(subset="sic",inplace=True)


# Data of a Single Company

In [ ]:
var_co_name = "SALES"
df_ANNUAL_temp = df_ANNUAL.copy()
df_co_data = df_ANNUAL_temp[df_ANNUAL_temp["name"].str.contains(var_co_name)]
df_ANNUAL_temp = pd.DataFrame()
df_co_data.drop_duplicates(subset=["name"],inplace=True)
df_co_data


In [ ]:
# Get the data of the Foreign company 
var_co_name = "SFDC"
var_cik = "1108524"
df_ANNUAL_temp = df_ANNUAL.copy()
df_co = df_ANNUAL_temp[df_ANNUAL_temp["cik"].str.contains(var_cik)]
df_ANNUAL_temp = pd.DataFrame()
print(len(df_co))
df_co.to_csv((folder_cos+var_co_name+" "+var_cik+" ANNUAL.csv"))

# Financial Statement Entries

In [ ]:
# Income Statement
var_FS = var_IS
get_financial_statement_elements(var_FS)

var_FS = var_BS
get_financial_statement_elements(var_FS)

var_FS = var_CF
get_financial_statement_elements(var_FS)


In [ ]:
def get_financial_statement_elements(var_FS):
    # GET METRICs 
    ls_met_tags = pd.unique(df_all_tags["metric"][(df_all_tags["stmt"] == var_FS)])

#df_all_tags_copy = df_all_tags.copy()
    counter_met = 0

    while counter_met < len(ls_met_tags):

    # GET RELEVANT TAGS    
        df_rel_tags = df_all_tags[(df_all_tags["metric"] == ls_met_tags[counter_met])&((df_all_tags["stmt"] == var_FS))]

        df_rel_tags.drop_duplicates(inplace=True)
        df_rel_tags = pd.DataFrame(df_rel_tags)
        print(("Tags in Metric "+ls_met_tags[counter_met]+" are "+np.str(len(df_rel_tags))))

    # PARE DOWN THE FINANCIAL STATEMENT DATA TO ONLY THOSE TAGS THAT ARE RELEVANT
        df_ANNUAL_temp = df_ANNUAL.copy()
        
        if var_FS == var_BS:
            df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS)
                                           &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))]
        else:
            df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS) 
                                       &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))
                                   &(df_ANNUAL_temp["qtrs"] == "4")]

        print(("Entries with "+ls_met_tags[counter_met]+" relevant tags are "+np.str(len(df_ANNUAL_rel))))

        ls_rel_tags = pd.unique(df_rel_tags["tag"])

        df_ANNUAL_temp = df_ANNUAL_rel.copy()
        df_rel_cos = df_ANNUAL_temp[["cfy"]]
        df_rel_cos.drop_duplicates(subset=["cfy"],inplace=True)
        df_ANNUAL_temp = pd.DataFrame()
        df_rel_cos_temp = df_rel_cos.copy()
        print(("Number of Relevant Co-FY Combos "+np.str(len(df_rel_cos_temp))))

        # Create a copy of the relevant entries
        df_ANNUAL_rcop = df_ANNUAL_rel.copy()

        df_entries_temp = pd.DataFrame()
        df_cosfy_tagfound = pd.DataFrame()
        df_cosfy_alltagfound = pd.DataFrame()
        df_cf_list = pd.DataFrame()
        counter_tag = 0

        while counter_tag < len(ls_rel_tags):

            df_entries_temp = df_ANNUAL_rcop[df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]
            df_cosfy_tagfound = df_entries_temp[(df_entries_temp["tag"] == ls_rel_tags[counter_tag])]

            df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)

            df_cf_list = df_cosfy_tagfound[["cfy"]]
            df_cf_list.drop_duplicates()
    #   df_cf_list["cik-fy"] = df_cf_list["cik"] + "-" + df_cf_list["fy"]    
            df_rel_cos_temp = df_rel_cos_temp[~(df_rel_cos_temp["cfy"].isin(df_cf_list["cfy"]))]

            print((ls_rel_tags[counter_tag]+ " entries "+np.str(len(df_cosfy_tagfound))))

            counter_tag = counter_tag + 1

        df_cosfy_alltagfound = pd.merge(df_cosfy_alltagfound,df_currency,left_on="uom",right_on="Currency",how="left")
        df_cosfy_alltagfound["value Bn USD"] = df_cosfy_alltagfound["value"] / df_cosfy_alltagfound["inUSD"] / 1000000000

        # MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
        df_ANN_met_found = pd.merge(df_cosfy_alltagfound,df_sic_data,on="sic",how="left")

        print("Entries Found "+np.str(len(df_ANN_met_found)))
        df_ANN_met_found.to_csv((folder_ann+var_FS+"/"+ls_met_tags[counter_met]+".csv"))

        df_ANN_fnd_cik_fy = df_ANN_met_found["cfy"]
        df_ANN_fnd_cik_fy.drop_duplicates(inplace=True)
        df_ANN_fnd_cik_fy = pd.DataFrame(df_ANN_fnd_cik_fy)
        print("CIK-FY Tag Found "+np.str(len(df_ANN_fnd_cik_fy)))

        df_ANN_all_cik_fy = df_ANNUAL["cfy"]
        df_ANN_all_cik_fy.drop_duplicates(inplace=True)
        df_ANN_all_cik_fy = pd.DataFrame(df_ANN_all_cik_fy)
        print("CIK-FY All "+np.str(len(df_ANN_all_cik_fy)))

        df_ANN_cik_missing = df_ANN_all_cik_fy[~(df_ANN_all_cik_fy["cfy"].isin(df_ANN_fnd_cik_fy["cfy"]))]
        print("CIK-FY Missing "+np.str(len(df_ANN_cik_missing)))

    # GET THE IRRELEVANT TAGS
        df_irl_tags = df_all_tags["tag"][~(df_all_tags["metric"] == ls_met_tags[counter_met])&(df_all_tags["stmt"] == var_FS)]
        df_irl_tags.drop_duplicates(inplace=True)
        df_irl_tags = pd.DataFrame(df_irl_tags)

    # GET THE GENERIC TAGS
        df_gnr_tags = df_all_tags["tag_generic"][(df_all_tags["metric"] == ls_met_tags[counter_met])&((df_all_tags["stmt"] == var_FS))]
        df_gnr_tags.drop_duplicates(inplace=True)
        df_gnr_tags = pd.DataFrame(df_gnr_tags)
        df_gnr_tags.rename(columns={"tag_generic":"tag"},inplace=True)

    # Found Generic Tag
        df_ANN_src = df_ANNUAL.copy()

        df_found_generic = df_ANN_src[(df_ANN_src["stmt"] == var_FS)
                                      &(df_ANN_src["qtrs"] == "4")&(df_ANN_src["cfy"].isin(df_ANN_cik_missing["cfy"]))
                                      &(df_ANN_src["tag"].str.contains("|".join(df_gnr_tags["tag"])))]

        df_found_generic.drop_duplicates(subset=["tag"],inplace=True)
        print("Found Generic "+np.str(len(df_found_generic)))
        df_found_generic.to_csv((folder_ann+"/GEN/"+var_FS+" "+ls_met_tags[counter_met]+" GENERIC FND.csv"))

        counter_met = counter_met + 1

    return

# Balance Sheet Cash
Cash entries are additive and need a special code


In [ ]:
# GET METRICs 
var_rel_met = "CASH"
var_FS = var_BS
ls_met_tags = pd.unique(df_all_tags["metric"][(df_all_tags["stmt"] == var_FS)&( df_all_tags["metric"]== var_rel_met)])
counter_met = 0

while counter_met < len(ls_met_tags):

# GET RELEVANT TAGS    
    df_rel_tags = df_all_tags[(df_all_tags["metric"] == ls_met_tags[counter_met])&((df_all_tags["stmt"] == var_FS))]
    df_addln_tags = pd.DataFrame()
    df_rel_tags.drop_duplicates(inplace=True)
    df_rel_tags = pd.DataFrame(df_rel_tags)
    print(("Tags in Metric "+ls_met_tags[counter_met]+" are "+np.str(len(df_rel_tags))))

# PARE DOWN THE FINANCIAL STATEMENT DATA TO ONLY THOSE TAGS THAT ARE RELEVANT
    df_ANNUAL_temp = df_ANNUAL.copy()

    if var_FS == var_BS:
        df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS)
                                       &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))]
    else:
        df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS) 
                                   &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))
                               &(df_ANNUAL_temp["qtrs"] == "4")]

    print(("Entries with "+ls_met_tags[counter_met]+" relevant tags are "+np.str(len(df_ANNUAL_rel))))

# We are removing the entries from the loop but will add them back in for every iteration.
# This helps ensure that multiple entries for CASH will get picked up
    li_cash_tags = ["MARKETABLESECURITIESCURRENT","AVAILABLEFORSALESECURITIESCURRENT","SHORTTERMINVESTMENTS"]
    df_rel_tags = df_rel_tags[~(df_rel_tags["tag"].isin(li_cash_tags))]

    ls_rel_tags = pd.unique(df_rel_tags["tag"])

    df_ANNUAL_temp = df_ANNUAL_rel.copy()
    df_rel_cos = df_ANNUAL_temp[["cfy"]]
    df_rel_cos.drop_duplicates(subset=["cfy"],inplace=True)
    df_ANNUAL_temp = pd.DataFrame()
    df_rel_cos_temp = df_rel_cos.copy()
    print(("Number of Relevant Co-FY Combos "+np.str(len(df_rel_cos_temp))))

    # Create a copy of the relevant entries
    df_ANNUAL_rcop = df_ANNUAL_rel.copy()

    df_entries_temp = pd.DataFrame()
    df_cosfy_tagfound = pd.DataFrame()
    df_cosfy_alltagfound = pd.DataFrame()
    df_cf_list = pd.DataFrame()
    counter_tag = 0

    while counter_tag < len(ls_rel_tags):

# Adding back tge deleted cash entries        
        ls_rel_tags2 = np.append(ls_rel_tags[counter_tag],li_cash_tags)

        print(ls_rel_tags2)

        df_entries_temp = df_ANNUAL_rcop[df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]
        df_cosfy_tagfound = df_entries_temp[(df_entries_temp["tag"].isin(ls_rel_tags2))]

        df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)

        df_cf_list = df_cosfy_tagfound[["cfy"]]
        df_cf_list.drop_duplicates()
#   df_cf_list["cik-fy"] = df_cf_list["cik"] + "-" + df_cf_list["fy"]    
        df_rel_cos_temp = df_rel_cos_temp[~(df_rel_cos_temp["cfy"].isin(df_cf_list["cfy"]))]

        print((ls_rel_tags[counter_tag]+ " entries "+np.str(len(df_cosfy_tagfound))))

        counter_tag = counter_tag + 1

    df_cosfy_alltagfound = pd.merge(df_cosfy_alltagfound,df_currency,left_on="uom",right_on="Currency",how="left")
    df_cosfy_alltagfound["value Bn USD"] = df_cosfy_alltagfound["value"] / df_cosfy_alltagfound["inUSD"] / 1000000000

    # MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
    df_ANN_met_found = pd.merge(df_cosfy_alltagfound,df_sic_data,on="sic",how="left")

    print("Entries Found "+np.str(len(df_ANN_met_found)))
    df_ANN_met_found.to_csv((folder_ann+var_FS+"/"+ls_met_tags[counter_met]+".csv"))

    df_ANN_fnd_cik_fy = df_ANN_met_found["cfy"]
    df_ANN_fnd_cik_fy.drop_duplicates(inplace=True)
    df_ANN_fnd_cik_fy = pd.DataFrame(df_ANN_fnd_cik_fy)
    print("CIK-FY Tag Found "+np.str(len(df_ANN_fnd_cik_fy)))

    df_ANN_all_cik_fy = df_ANNUAL["cfy"]
    df_ANN_all_cik_fy.drop_duplicates(inplace=True)
    df_ANN_all_cik_fy = pd.DataFrame(df_ANN_all_cik_fy)
    print("CIK-FY All "+np.str(len(df_ANN_all_cik_fy)))

    df_ANN_cik_missing = df_ANN_all_cik_fy[~(df_ANN_all_cik_fy["cfy"].isin(df_ANN_fnd_cik_fy["cfy"]))]
    print("CIK-FY Missing "+np.str(len(df_ANN_cik_missing)))

# GET THE IRRELEVANT TAGS
    df_irl_tags = df_all_tags["tag"][~(df_all_tags["metric"] == ls_met_tags[counter_met])&(df_all_tags["stmt"] == var_FS)]
    df_irl_tags.drop_duplicates(inplace=True)
    df_irl_tags = pd.DataFrame(df_irl_tags)

# GET THE GENERIC TAGS
    df_gnr_tags = df_all_tags["tag_generic"][(df_all_tags["metric"] == ls_met_tags[counter_met])&((df_all_tags["stmt"] == var_FS))]
    df_gnr_tags.drop_duplicates(inplace=True)
    df_gnr_tags = pd.DataFrame(df_gnr_tags)
    df_gnr_tags.rename(columns={"tag_generic":"tag"},inplace=True)

# Found Generic Tag
    df_ANN_src = df_ANNUAL.copy()

    df_found_generic = df_ANN_src[(df_ANN_src["stmt"] == var_FS)
                                  &(df_ANN_src["qtrs"] == "4")&(df_ANN_src["cfy"].isin(df_ANN_cik_missing["cfy"]))
                                  &(df_ANN_src["tag"].str.contains("|".join(df_gnr_tags["tag"])))]

    df_found_generic.drop_duplicates(subset=["tag"],inplace=True)
    print("Found Generic "+np.str(len(df_found_generic)))
    df_found_generic.to_csv((folder_ann+"/GEN/"+var_FS+" "+ls_met_tags[counter_met]+" GENERIC FND.csv"))

    counter_met = counter_met + 1

In [ ]:
df_nc_tags.head(2)

# CASH FLOW NET CASH

In [ ]:
var_FS = var_CF
var_rel_met = "NETCASH"
df_cosfy_alltagfound = pd.DataFrame()

df_ANNUAL_temp = df_ANNUAL.copy()
ls_opr = ["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIES","NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIESCONTINUINGOPERATIONS"]
df_cosfy_tagfound = df_ANNUAL_temp[(df_ANNUAL_temp["tag"].isin(ls_opr))]
df_cosfy_tagfound.sort_values(by=["cfy","value"],inplace=True)
df_cosfy_tagfound.drop_duplicates(subset="cfy",inplace=True)
df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)
print((var_rel_met+ " OPRT entries "+np.str(len(df_cosfy_tagfound))))


df_ANNUAL_temp = df_ANNUAL.copy()
ls_inv = ["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIES","NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIESCONTINUINGOPERATIONS"]
df_cosfy_tagfound = df_ANNUAL_temp[(df_ANNUAL_temp["tag"].isin(ls_inv))]
df_cosfy_tagfound.sort_values(by=["cfy","value"],inplace=True)
df_cosfy_tagfound.drop_duplicates(subset="cfy",inplace=True)
df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)
print((var_rel_met+ " INVST entries "+np.str(len(df_cosfy_tagfound))))

df_ANNUAL_temp = df_ANNUAL.copy()
ls_fin = ["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIES","NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIESCONTINUINGOPERATIONS"]
df_cosfy_tagfound = df_ANNUAL_temp[(df_ANNUAL_temp["tag"].isin(ls_fin))]
df_cosfy_tagfound.sort_values(by=["cfy","value"],inplace=True)
df_cosfy_tagfound.drop_duplicates(subset="cfy",inplace=True)
df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)
print((var_rel_met+ " FINANC entries "+np.str(len(df_cosfy_tagfound))))

df_cosfy_alltagfound = pd.merge(df_cosfy_alltagfound,df_currency,left_on="uom",right_on="Currency",how="left")
df_cosfy_alltagfound["value Bn USD"] = df_cosfy_alltagfound["value"] / df_cosfy_alltagfound["inUSD"] / 1000000000

# MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
df_ANN_met_found = pd.merge(df_cosfy_alltagfound,df_sic_data,on="sic",how="left")
    
print("Entries Found "+np.str(len(df_ANN_met_found)))
df_ANN_met_found.to_csv((folder_ann+var_FS+"/"+var_rel_met+".csv"))

df_ANN_fnd_cik_fy = df_ANN_met_found["cfy"]
df_ANN_fnd_cik_fy.drop_duplicates(inplace=True)
df_ANN_fnd_cik_fy = pd.DataFrame(df_ANN_fnd_cik_fy)
print("CIK-FY Tag Found "+np.str(len(df_ANN_fnd_cik_fy)))

df_ANN_all_cik_fy = df_ANNUAL["cfy"]
df_ANN_all_cik_fy.drop_duplicates(inplace=True)
df_ANN_all_cik_fy = pd.DataFrame(df_ANN_all_cik_fy)
print("CIK-FY All "+np.str(len(df_ANN_all_cik_fy)))

df_ANN_cik_missing = df_ANN_all_cik_fy[~(df_ANN_all_cik_fy["cfy"].isin(df_ANN_fnd_cik_fy["cfy"]))]
print("CIK-FY Missing "+np.str(len(df_ANN_cik_missing)))
    
# GET THE IRRELEVANT TAGS
df_irl_tags = df_all_tags["tag"][~(df_all_tags["metric"] == var_rel_met)&(df_all_tags["stmt"] == var_FS)]
df_irl_tags.drop_duplicates(inplace=True)
df_irl_tags = pd.DataFrame(df_irl_tags)

# GET THE GENERIC TAGS
df_gnr_tags = df_all_tags["tag_generic"][(df_all_tags["metric"] == var_rel_met)&((df_all_tags["stmt"] == var_FS))]
df_gnr_tags.drop_duplicates(inplace=True)
df_gnr_tags = pd.DataFrame(df_gnr_tags)
df_gnr_tags.rename(columns={"tag_generic":"tag"},inplace=True)

# Found Generic Tag
df_all_10K_src = df_ANNUAL.copy()
df_found_generic = df_all_10K_src[(df_all_10K_src["stmt"] == var_FS)
                                  &(df_all_10K_src["qtrs"] == "4")&(df_all_10K_src["cfy"].isin(df_ANN_cik_missing["cfy"]))
                                  &(df_all_10K_src["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
df_found_generic.drop_duplicates(subset=["tag"],inplace=True)
print("Found Generic "+np.str(len(df_found_generic)))
df_found_generic.to_csv((folder_ann+"/GEN/"+var_FS+" "+var_rel_met+" GENERIC FND.csv"))

